In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Feature Engineering

In [2]:
dff = pd.read_csv("data_class.csv")


In [3]:
dff

,Unnamed: 0,date,id,activity,circumplex.arousal,circumplex.valence,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,day_2,day_3,day_4,day_5,day_6,month_2,month_3,month_4,month_5,month_6
0,0,2014-02-26,AS14.01,0.090464,-0.250000,0.75,0.000,0.000,0.000,0.000,...,1,0,0,0,0,1,0,0,0,0
1,1,2014-03-21,AS14.01,0.134050,0.200000,0.20,1090.617,2850.042,100.898,49.544,...,0,0,1,0,0,0,1,0,0,0
2,2,2014-03-22,AS14.01,0.236880,0.600000,0.50,476.737,1946.683,4.010,21.076,...,0,0,0,1,0,0,1,0,0,0
3,3,2014-03-23,AS14.01,0.142741,0.200000,0.80,540.915,2680.535,4.028,43.403,...,0,0,0,0,1,0,1,0,0,0
4,4,2014-03-24,AS14.01,0.078961,0.800000,0.00,731.456,5070.628,70.289,34.106,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,1207,2014-05-26,AS14.33,0.115201,-0.400000,0.00,571.902,750.571,79.963,0.000,...,0,0,0,0,0,0,0,0,1,0
1208,1208,2014-05-27,AS14.33,0.012704,-0.600000,0.40,269.550,771.182,32.137,0.000,...,0,0,0,0,0,0,0,0,1,0
1209,1209,2014-05-28,AS14.33,0.103301,0.000000,1.20,1093.741,1057.568,153.121,0.000,...,1,0,0,0,0,0,0,0,1,0
1210,1210,2014-05-29,AS14.33,0.169354,-1.333333,1.00,248.063,198.112,5.018,0.000,...,0,1,0,0,0,0,0,0,1,0


In [4]:
dff.columns

Index(['Unnamed: 0', 'date', 'id', 'activity', 'circumplex.arousal',
       'circumplex.valence', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'call',
       'screen', 'sms', 'total_app_usage', 'total_app_time',
       'productivity_attribute', 'social_app',
       'productivity_to_social_app_ratio', 'social_phone', 'mood_quantiles',
       'activity_3day_avg', 'circumplex.arousal_3day_avg',
       'circumplex.valence_3day_avg', 'appCat.builtin_3day_avg',
       'appCat.communication_3day_avg', 'appCat.entertainment_3day_avg',
       'appCat.finance_3day_avg', 'appCat.game_3day_avg',
       'appCat.office_3day_avg', 'appCat.other_3day_avg',
       'appCat.social_3day_avg', 'appCat.travel_3day_avg',
       'appCat.unknown_3day_avg', 'appCat.utilities_3day_avg',
       'appCat.weather_3day_avg

# Feature Selection


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assuming 'dff' is your DataFrame and has been properly loaded and prepared
X = dff.drop(['id', 'date', 'mood_quantiles', 'Unnamed: 0'], axis=1)
y = dff['mood_quantiles']

# Initialize the scaler
scaler = MinMaxScaler()

# Scale the features (X)
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)  # Convert the scaled array back to a DataFrame

# Combine the scaled features with the target variable into a new DataFrame
df_scaled = pd.concat([X_scaled, y.reset_index(drop=True)], axis=1)

# Backwards Selection 

# RFECV opti

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import pandas as pd

# Assuming 'dff' is your DataFrame and 'X_scaled' and 'y' have been prepared as described previously
# Initialize the classifier with 50 estimators
tscv = TimeSeriesSplit(n_splits=5)
classifier = RandomForestClassifier(n_estimators=10, random_state=42)

# Create the RFE object with cross-validation
rfecv = RFECV(estimator=classifier, step=2, cv=tscv, scoring='accuracy')

# Fit RFECV to the scaled features and the target
rfecv.fit(X_scaled, y)

# List the best features: features with support_ = True are selected
selected_features = [X_scaled.columns[i] for i in range(X_scaled.shape[1]) if rfecv.support_[i]]

# Print the selected features
print("Selected features:", selected_features)


Selected features: ['activity', 'circumplex.valence', 'appCat.communication_3day_avg', 'circumplex.valence_7day_avg', 'mood_quantiles_7day_avg']


In [11]:
X_r = X_scaled[selected_features]

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# Assuming X_bs and y have been defined and preprocessed properly

# Initialize TimeSeriesSplit

# Determine the train and test indices from the last split
train_index, test_index = next(reversed(list(tscv.split(X_r))))
X_train, X_test = X_r.iloc[train_index], X_r.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Define models and parameters for GridSearch
models_params = {
    'DecisionTreeClassifier': (DecisionTreeClassifier(), {'criterion': ['gini', 'entropy'], 'random_state': [0]}),
    'RandomForestClassifier': (RandomForestClassifier(), {'n_estimators': [100, 150, 200], 'criterion': ['gini', 'entropy'], 'random_state': [0]}),
    'AdaBoostClassifier': (AdaBoostClassifier(), {'n_estimators': [100, 150, 200], 'learning_rate': [0.1, 0.5, 0.8, 1], 'random_state': [0]}),
    'GradientBoostingClassifier': (GradientBoostingClassifier(), {'n_estimators': [100, 150, 200], 'learning_rate': [0.1, 0.5, 0.8, 1], 'random_state': [0]}),
    'XGBClassifier': (XGBClassifier(), {'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 150, 200], 'random_state': [0], 'eval_metric': ['logloss', 'error']})
}

model_results = []

# Perform Grid Search for each model and test them
for model_name, (model, params) in models_params.items():
    grid = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=tscv, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    best_score = grid.best_score_
    
    # Train the best model from GridSearchCV
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = best_model.predict(X_test)
    
    # Evaluate the model performance on the test set
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: Test Accuracy = {test_accuracy * 100:.2f}%')
    
    model_results.append({
        'Model': model_name,
        'Best Score': f'{best_score * 100:.2f}%',
        'Test Accuracy': f'{test_accuracy * 100:.2f}%',
        'Best Parameters': best_params
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(model_results)
print(results_df)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
DecisionTreeClassifier: Test Accuracy = 60.89%
Fitting 5 folds for each of 6 candidates, totalling 30 fits
RandomForestClassifier: Test Accuracy = 69.80%
Fitting 5 folds for each of 12 candidates, totalling 60 fits
AdaBoostClassifier: Test Accuracy = 74.26%
Fitting 5 folds for each of 12 candidates, totalling 60 fits
GradientBoostingClassifier: Test Accuracy = 71.29%
Fitting 5 folds for each of 18 candidates, totalling 90 fits
XGBClassifier: Test Accuracy = 71.29%
                        Model Best Score Test Accuracy  \
0      DecisionTreeClassifier     52.02%        60.89%   
1      RandomForestClassifier     57.50%        69.80%   
2          AdaBoostClassifier     56.07%        74.26%   
3  GradientBoostingClassifier     57.86%        71.29%   
4               XGBClassifier     56.79%        71.29%   

                                     Best Parameters  
0           {'criterion': 'gini', 'random_state': 0}  
1  {'criteri

In [ ]:
results_df.to_csv('model_results.csv', index=False)


In [ ]:
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, precision_score, recall_score, auc, roc_curve, accuracy_score, f1_score, roc_auc_score

ax= plt.subplot()
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='g', ax=ax); 
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Q1', 'Q2','Q3']); ax.yaxis.set_ticklabels(['Q1', 'Q2','Q3']);